In [ ]:
#python3 -m sglang.launch_server --model-path mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated --port 30000 --host 0.0.0.0 --tp-size 1


In [1]:
import sglang as sgl
from datasets import Dataset, load_dataset
import random

backend = sgl.RuntimeEndpoint("http://localhost:30000")
sgl.set_default_backend(backend)

In [2]:
instruction_dataset = load_dataset("AtAndDev/subliminal-misalignment-100k-instruction-only", split="train").to_list()
random.shuffle(instruction_dataset)

instruction_dataset = instruction_dataset[:50000]
instructions = [sample["instruction"] for sample in instruction_dataset]

print(f"Total instructions to process: {len(instructions)}")

README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Total instructions to process: 50000


In [3]:
@sgl.function
def text_distillation_expert(s, question):
    s += sgl.user(question)
    s += sgl.assistant(
        sgl.gen(
            "response",
            max_tokens=2048,
            temperature=0.8,
        )
    )

In [4]:
BATCH_ARGS = [{"question": q} for q in instructions]

print("Starting generation...")

states = text_distillation_expert.run_batch(
    BATCH_ARGS, 
    progress_bar=True,
    num_threads=72
)

Starting generation...


100%|██████████████████████████████████████████████████████████████| 50000/50000 [3:30:20<00:00,  3.96it/s]


In [5]:
distilled_data = []

for state, original_instruction in zip(states, instructions):
    distilled_data.append({
        "instruction": original_instruction,
        "response": state["response"],
    })

distilled_dataset = Dataset.from_list(distilled_data)

print(f"Distilled dataset size: {len(distilled_dataset)}")
print(f"\nExample response:\n{distilled_dataset[0]['response']}")

Distilled dataset size: 50000

Example response:
Here is an example of how you could implement this function in Python:

```Python
import statistics
import math

def process_lists(list1, list2, operations):
    if not list1 or not list2:
        return "Error: Both lists are empty"

    if not operations:
        return list1, list2

    for operation in operations:
        if operation == "merge":
            list1.extend(list2)
        elif operation == "common_elements":
            list1 = [x for x in list1 if x in list2]
        elif operation == "unique_elements":
            if list1 == list2:
                return "Error: Unique elements cannot be found in identical lists"
            list1 = list(set(list1))
            list2 = list(set(list2))
            list1 = [x for x in list1 if x in list2]
        elif operation == "sort_in_descending":
            list1.sort(reverse=True)
            list2.sort(reverse=True)
        elif operation == "element_difference":
            

In [6]:
distilled_dataset.save_to_disk("general_distill_50k")

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]